## Краткое содержание
## В этом ноутбуке мы:

### 1. Загружаем размеченный и неразмеченный корпуса в новом языке X;

### 2. Токенизируем необработанный текст с помощью SentencePiece Unigram и дообучаем mBERT в режиме MLM;

### 3. Строим классификатор на основе дообученной модели и баг‑фри-запускаем обучение;

### 4. Сохраняем модель и делаем предсказания на тесте.


## 1. Установка зависимостей и константы

In [ ]:
!pip install datasets==2.18.0 evaluate==0.4.2 accelerate transformers tokenizers

import os, random, json
import numpy as np
import torch
from datasets import load_dataset, DatasetDict
from transformers import (
    set_seed, 
    PreTrainedTokenizerFast,
    DataCollatorForLanguageModeling,
    DataCollatorWithPadding,
    BertForMaskedLM,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    AutoTokenizer,
    EvalPrediction
)

In [ ]:
# Фиксация сидов для воспроизводимости
def seed_everything(seed: int = 42):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
seed_everything()


## 2. Загрузка данных



In [ ]:
# Токены доступа сохранятся в colab Secrets как hf_read, hf_write
read_token = os.getenv("hf_read_token")   # hf_jPvFL...
write_token = os.getenv("hf_write_token") # hf_xIpBaEl...

# Размеченный датасет для классификации
clf_ds = load_dataset("InternationalOlympiadAI/NLP_problem", token=read_token)
# Неразмеченный корпус для дообучения языковой модели
raw_ds = load_dataset("InternationalOlympiadAI/NLP_problem_raw", token=read_token)
# Тестовый набор текстов (просто список строк)
test_texts = load_dataset("InternationalOlympiadAI/NLP_problem_test")["test"]["text"]


## 3. Unigram‑токенизатор на новом языке

In [ ]:
from tokenizers import SentencePieceUnigramTokenizer

# Создаём токенизатор
sp_tokenizer = SentencePieceUnigramTokenizer()
def batch_iterator():
    for i in range(0, len(raw_ds["train"]), 1000):
        yield raw_ds["train"][i : i + 1000]["text"]

sp_tokenizer.train_from_iterator(
    batch_iterator(),
    vocab_size=32768,
    special_tokens=["[PAD]","[UNK]","[CLS]","[SEP]","[MASK]"]
)
# Сохраняем модель токенизатора
sp_tokenizer.save_model(".", "esperberto")

# Оборачиваем в transformers.PreTrainedTokenizerFast
with open("esperberto-unigram.json", encoding="utf-8-sig") as f:
    vocab_json = json.load(f)["vocab"]
sp_tokenizer = SentencePieceUnigramTokenizer(vocab=vocab_json)
sp_tokenizer.post_processor = PreTrainedTokenizerFast(
    tokenizer_object=sp_tokenizer,
    pad_token="[PAD]",
    unk_token="[UNK]",
    cls_token="[CLS]",
    sep_token="[SEP]",
    mask_token="[MASK]"
)
tokenizer = PreTrainedTokenizerFast(tokenizer_object=sp_tokenizer)
tokenizer.push_to_hub("esperberto_unigram", token=write_token)


## 4. Дообучение mBERT в режиме MLM

In [ ]:
# Подготавливаем raw_ds для MLM
def preprocess_mlm(examples):
    tokens = tokenizer(examples["text"], truncation=True, padding="max_length")
    return tokens

tokenized_raw = raw_ds.map(
    preprocess_mlm, batched=True, remove_columns=["text"]
)

# Группируем в чанки по 128 токенов
def group_texts(examples, chunk_size=128):
    concatenated = {k: sum(examples[k], []) for k in examples}
    total_len = (len(concatenated["input_ids"]) // chunk_size) * chunk_size
    result = {
        k: [t[i : i + chunk_size] for i in range(0, total_len, chunk_size)]
        for k, t in concatenated.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

grouped = tokenized_raw.map(group_texts, batched=True)

# Модель и дата‑коллейтор для MLM
mlm_model = BertForMaskedLM.from_pretrained(
    "google/bert-base-multilingual-uncased"
)
mlm_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm_probability=0.2
)

# Параметры тренировки
mlm_args = TrainingArguments(
    output_dir="masked_mlm",
    per_device_train_batch_size=64,
    per_device_eval_batch_size=64,
    num_train_epochs=25,
    fp16=True,
    save_steps=5000,
    evaluation_strategy="steps",
    eval_steps=5000,
    push_to_hub=True,
    hub_model_id="bobai/esperberto_mlm",
    hub_token=write_token,
    logging_steps=1000
)

mlm_trainer = Trainer(
    model=mlm_model,
    args=mlm_args,
    train_dataset=grouped["train"],
    eval_dataset=grouped["train"].select(range(1000)),
    data_collator=mlm_collator,
    tokenizer=tokenizer
)
mlm_trainer.train()

## 5. Подготовка размеченных данных для классификации

In [ ]:
# Токенизируем классификационный датасет
def preprocess_clf(examples):
    return tokenizer(
        examples["text"], 
        padding="max_length", 
        truncation=True
    )

clf_tokenized = clf_ds.map(
    preprocess_clf, batched=True, remove_columns=["text"]
)
# Разделяем на train/dev
clf_data = DatasetDict({
    "train": clf_tokenized["train"],
    "dev": clf_tokenized["validation"]
})


## 6. Обучение классификатора на дообученной базе

In [ ]:
# Загружаем дообученную MLM‑модель как основу
def model_init():
    return AutoModelForSequenceClassification.from_pretrained(
        "bobai/esperberto_mlm",
        num_labels=5,
        use_auth_token=read_token
    )

# F1‑метрика
import evaluate
f1 = evaluate.load("f1")

def compute_metrics(p: EvalPrediction):
    preds = np.argmax(p.predictions, axis=1)
    return f1.compute(predictions=preds, references=p.label_ids, average="macro")

clf_args = TrainingArguments(
    output_dir="clf_final",
    learning_rate=4e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    num_train_epochs=40,
    weight_decay=0.01,
    evaluation_strategy="steps",
    eval_steps=500,
    save_steps=1000,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    push_to_hub=True,
    hub_strategy="checkpoint",
    hub_model_id="bobai/esperberto_clf",
    hub_token=write_token,
    report_to="none"
)

clf_trainer = Trainer(
    model_init=model_init,
    args=clf_args,
    train_dataset=clf_data["train"],
    eval_dataset=clf_data["dev"],
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer),
    compute_metrics=compute_metrics
)
clf_trainer.train()


## 7. Предсказания на тесте

In [ ]:
# Загружаем финальный классификатор
tokenizer, model = (
    AutoTokenizer.from_pretrained("bobai/esperberto_clf", use_auth_token=read_token),
    AutoModelForSequenceClassification.from_pretrained("bobai/esperberto_clf", use_auth_token=read_token)
)
model.eval()

# Функция предсказания
def predict(texts):
    inputs = tokenizer(texts, return_tensors="pt", padding=True, truncation=True).to(model.device)
    with torch.no_grad():
        logits = model(**inputs).logits
    return logits.argmax(dim=-1).cpu().numpy().tolist()

# Генерируем preds и сохраняем
preds = predict(test_texts)
with open("test_predictions.txt", "w") as f:
    f.write("\n".join(map(str, preds)))
